In [37]:
import re

def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove punctuation (but keep spaces)
    text = re.sub(r'\s+', ' ', text).strip()    # Normalize multiple spaces to one
    return text



In [32]:
import pandas as pd

real = pd.read_csv("../data/raw/True.csv")
fake = pd.read_csv("../data/raw/Fake.csv")

real['label'] = "REAL"
fake["label"] = "FAKE"

In [40]:
cols = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']

train = pd.read_csv("../data/raw/train.tsv",sep="\t",header=None,names=cols)
test = pd.read_csv("../data/raw/test.tsv",sep="\t",header=None,names=cols)
valid = pd.read_csv("../data/raw/valid.tsv",sep="\t",header=None,names=cols)

liar = pd.concat([train,test,valid]).reset_index(drop=True)

fake_labels = ['pants-fire', 'false', 'barely-true']
real_labels = ['half-true', 'mostly-true', 'true']

liar['label'] = liar['label'].apply(lambda x: 'FAKE' if x in fake_labels else 'REAL')

liar['text'] = liar['statement'].apply(clean_text)
liar = liar[['text','label']]


# Save
liar.to_csv("../data/processed/liar_clean.csv", index=False)

In [34]:
df = pd.concat([real,fake]).reset_index(drop=True)
df['text'] = (df['title']+" "+df["text"]).apply(clean_text)
df = df[["text","label"]]

In [35]:
df.head()

,text,label
0,as us budget fight looms republicans flip thei...,REAL
1,us military to accept transgender recruits on ...,REAL
2,senior us republican senator let mr mueller do...,REAL
3,fbi russia probe helped by australian diplomat...,REAL
4,trump wants postal service to charge much more...,REAL


In [36]:
df.info()
df['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   label   44898 non-null  object
dtypes: object(2)
memory usage: 701.7+ KB


label
FAKE    23481
REAL    21417
Name: count, dtype: int64